In [23]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

def extract_batting_data(series_id, match_id):

    URL = 'https://www.espncricinfo.com/series/'+ str(series_id) + '/scorecard/' + str(match_id)
    page = requests.get(URL)
    bs = BeautifulSoup(page.content, 'lxml')

    table_body=bs.find_all('tbody')
    batsmen_df = pd.DataFrame(columns=["Name","Desc","Runs", "Balls", "4s", "6s", "SR", "Team"])
    for i, table in enumerate(table_body[0:4:2]):
        rows = table.find_all('tr')
        for row in rows[::2]:
            cols=row.find_all('td')
            cols=[x.text.strip() for x in cols]
            if cols[0] == 'Extras':
                continue
            if len(cols) > 7:
                batsmen_df = batsmen_df.append(pd.Series(
                [re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip(), cols[1], 
                cols[2], cols[3], cols[5], cols[6], cols[7], i+1], 
                index=batsmen_df.columns ), ignore_index=True)
            else:
                batsmen_df = batsmen_df.append(pd.Series(
                [re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip(), cols[1], 
                0, 0, 0, 0, 0, i+1], index = batsmen_df.columns), ignore_index=True)
                
    for i in range(2):
        dnb_row = bs.find_all("tfoot")[i].find_all("div")
        for c in dnb_row:
            dnb_cols = c.find_all('span')
            dnb = [x.text.strip().split("(c)")[0] for x in dnb_cols]
            dnb = filter(lambda item: item, [re.sub(r"\W+", ' ', x).strip() for x in dnb])
            for dnb_batsman in dnb:
                batsmen_df = batsmen_df.append(pd.Series([dnb_batsman, "DNB", 0, 0, 0, 0, 0, i+1], index = batsmen_df.columns), ignore_index =True)

    return batsmen_df

In [24]:
extract_batting_data(series_id = 8048, match_id = 1136561)

Name                             Desc Runs Balls  \
0                    Rohit Sharma                c Rayudu b Watson   15    18   
1                      Evin Lewis                     lbw b Chahar    0     2   
2                    Ishan Kishan             c Wood b Imran Tahir   40    29   
3                Suryakumar Yadav       c Harbhajan Singh b Watson   43    29   
4                   Hardik Pandya                          not out   22    20   
5                   Krunal Pandya                          not out   41    22   
6                    Shane Watson              c Lewis b HH Pandya   16    14   
7                   Ambati Rayudu                   lbw b Markande   22    19   
8                    Suresh Raina          c KH Pandya b HH Pandya    4     6   
9                    Kedar Jadhav                          not out   24    22   
10                       MS Dhoni                   lbw b Markande    5     5   
11                Ravindra Jadeja      c Yadav b Mustafizur Rahman   12    13   
12                   Dwayne Bravo                c Sharma b Bumrah   68    30   
13                  Deepak Chahar      st †Ishan Kishan b Markande    0     1   
14                Harbhajan Singh           c Bumrah b McClenaghan    8     5   
15                      Mark Wood  c Mustafizur Rahman b HH Pandya    1     3   
16                    Imran Tahir                          not out    2     2   
17                 Kieron Pollard                              DNB    0     0   
18                 Jasprit Bumrah                              DNB    0     0   
19           Mitchell McClenaghan                              DNB    0     0   
20              Mustafizur Rahman                              DNB    0     0   
21                Mayank Markande                              DNB    0     0   
22          1 7 Evin Lewis 2 1 ov                              DNB    0     0   
23       2 20 Rohit Sharma 3 5 ov                              DNB    0     0   
24  3 98 Suryakumar Yadav 12 3 ov                              DNB    0     0   
25     4 113 Ishan Kishan 14 4 ov                              DNB    0     0   
26       1 27 Shane Watson 3 4 ov                              DNB    0     0   
27       2 42 Suresh Raina 5 6 ov                              DNB    0     0   
28      3 42 Ambati Rayudu 6 3 ov                              DNB    0     0   
29           4 51 MS Dhoni 8 3 ov                              DNB    0     0   
30   5 75 Ravindra Jadeja 11 6 ov                              DNB    0     0   
31                           5 83                              DNB    0     0   
32     6 84 Deepak Chahar 12 6 ov                              DNB    0     0   
33  7 105 Harbhajan Singh 14 5 ov                              DNB    0     0   
34        8 118 Mark Wood 16 3 ov                              DNB    0     0   
35     9 159 Dwayne Bravo 18 6 ov                              DNB    0     0   

   4s 6s      SR Team  
0   1  1   83.33    1  
1   0  0    0.00    1  
2   4  1  137.93    1  
3   6  1  148.27    1  
4   2  0  110.00    1  
5   5  2  186.36    1  
6   1  1  114.28    2  
7   4  0  115.78    2  
8   0  0   66.66    2  
9   1  2  109.09    2  
10  0  0  100.00    2  
11  1  0   92.30    2  
12  3  7  226.66    2  
13  0  0    0.00    2  
14  1  0  160.00    2  
15  0  0   33.33    2  
16  0  0  100.00    2  
17  0  0       0    1  
18  0  0       0    1  
19  0  0       0    1  
20  0  0       0    1  
21  0  0       0    1  
22  0  0       0    1  
23  0  0       0    1  
24  0  0       0    1  
25  0  0       0    1  
26  0  0       0    2  
27  0  0       0    2  
28  0  0       0    2  
29  0  0       0    2  
30  0  0       0    2  
31  0  0       0    2  
32  0  0       0    2  
33  0  0       0    2  
34  0  0       0    2  
35  0  0       0    2

In [25]:
def extract_bowling_data(series_id, match_id):

    URL = 'https://www.espncricinfo.com/series/'+ str(series_id) + '/scorecard/' + str(match_id)
    page = requests.get(URL)
    bs = BeautifulSoup(page.content, 'lxml')

    table_body=bs.find_all('tbody')
    bowler_df = pd.DataFrame(columns=['Name', 'Overs', 'Maidens', 'Runs', 'Wickets',
                                      'Econ', 'Dots', '4s', '6s', 'Wd', 'Nb','Team'])
    for i, table in enumerate(table_body[1:4:2]):
        rows = table.find_all('tr')
        for row in rows:
            cols=row.find_all('td')
            cols=[x.text.strip() for x in cols]
            bowler_df = bowler_df.append(pd.Series([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], 
                                                    cols[6], cols[7], cols[8], cols[9], cols[10], (i==0)+1], 
                                                   index=bowler_df.columns ), ignore_index=True)
    return bowler_df

In [26]:
extract_bowling_data(series_id = 8048, match_id = 1136561)

Name Overs Maidens Runs Wickets   Econ Dots 4s 6s Wd Nb  \
0          Deepak Chahar     3       0   14       1   4.66   12  2  0  0  0   
1           Shane Watson     4       0   29       2   7.25   12  1  2  1  0   
2        Harbhajan Singh     2       0   14       0   7.00    3  1  0  0  0   
3        Ravindra Jadeja     1       0    9       0   9.00    3  2  0  0  0   
4              Mark Wood     4       0   49       0  12.25    6  6  2  1  0   
5            Imran Tahir     2       0   23       1  11.50    4  3  1  0  0   
6           Dwayne Bravo     4       0   25       0   6.25    9  3  0  0  0   
7   Mitchell McClenaghan     4       0   44       1  11.00    5  4  2  1  0   
8      Mustafizur Rahman   3.5       0   39       1  10.17    8  6  1  0  0   
9         Jasprit Bumrah     4       0   37       1   9.25   10  0  4  0  1   
10         Hardik Pandya     4       0   24       3   6.00   12  1  1  3  0   
11       Mayank Markande     4       0   23       3   5.75   11  0  2  0  0   

   Team  
0     2  
1     2  
2     2  
3     2  
4     2  
5     2  
6     2  
7     1  
8     1  
9     1  
10    1  
11    1